# 03. モデル評価

このノートブックでは、学習済みモデルの包括的な評価を行います。

## 処理内容
1. 学習済みモデルの読み込み
2. テストデータでの評価
3. 詳細メトリクス計算
4. 統計的分析
5. ケース別分析
6. 結果保存

In [ ]:
# 必要なライブラリのインポート
import sys
from pathlib import Path

# プロジェクトルートをパスに追加
project_root = Path.cwd().parent
sys.path.append(str(project_root))

from config.experiment_configs import (
    get_baseline_config, get_cvae_config, get_gnn_config,
    get_large_model_config
)
from src.evaluation.evaluator import RIMDEvaluator, ModelComparator
from src.data.dataset import RIMDDataModule
from src.utils.experiment_manager import ExperimentManager
import logging
import json
import numpy as np
import pandas as pd

# ログレベル設定
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

## 評価設定の選択

評価したいモデルの実験設定を選択してください：

In [ ]:
# ===========================================
# 評価対象実験選択（学習済みモデルに対応）
# ===========================================

# 1. ベースライン実験評価
config = get_baseline_config()

# 2. CVAE実験評価
# config = get_cvae_config()

# 3. GNN実験評価
# config = get_gnn_config()

# 4. 大規模モデル実験評価
# config = get_large_model_config()

# ===========================================
# カスタム評価設定
# ===========================================
# config = get_baseline_config()
# config.exp_id = "custom_experiment"  # 評価したい実験IDに変更

print(f"評価対象実験: {config.exp_id}")
print(f"説明: {config.description}")
print(f"モデルタイプ: {config.model.model_type}")

In [ ]:
# 実験マネージャーの作成
exp_manager = ExperimentManager(config)

print(f"実験ディレクトリ: {exp_manager.exp_dir}")
print(f"使用デバイス: {exp_manager.device}")

# 学習済みモデルの存在確認
best_model_path = exp_manager.exp_dir / "models" / "best_model.pth"
latest_model_path = exp_manager.exp_dir / "models" / "latest_model.pth"

if best_model_path.exists():
    model_path = best_model_path
    print(f"✅ 最良モデルを使用: {model_path}")
elif latest_model_path.exists():
    model_path = latest_model_path
    print(f"✅ 最新モデルを使用: {model_path}")
else:
    print("❌ 学習済みモデルが見つかりません。02_training.ipynb を先に実行してください。")
    raise FileNotFoundError("No trained model found")

In [ ]:
# スケーラの読み込み
try:
    scalers = exp_manager.load_scalers()
    print("✅ スケーラ読み込み完了")
except FileNotFoundError:
    print("❌ スケーラが見つかりません。01_preprocessing.ipynb を先に実行してください。")
    raise

In [ ]:
# データモジュールの作成
datamodule = RIMDDataModule(config.data, scalers)

print("データセット情報:")
print(f"  Train: {len(datamodule.train_dataset)} cases")
print(f"  Val: {len(datamodule.val_dataset)} cases")
print(f"  Test: {len(datamodule.test_dataset)} cases")

## 単一モデル評価

選択したモデルの包括的評価を実行します。

In [ ]:
# 評価器の作成
evaluator = RIMDEvaluator(config, exp_manager)
print("✅ 評価器作成完了")

In [ ]:
# 包括的評価実行
print("=== モデル評価開始 ===")
try:
    evaluation_results = evaluator.evaluate_model(
        model_path=str(model_path),
        datamodule=datamodule,
        splits=['train', 'val', 'test'],  # 全分割で評価
        save_predictions=True
    )
    
    print("\n=== 評価完了 ===")
    print("評価結果が保存されました。")
    
except Exception as e:
    print(f"\n❌ 評価エラー: {e}")
    raise

## 評価結果の表示

計算された評価メトリクスを確認します。

In [ ]:
# 評価サマリーの表示
summary = evaluation_results['summary']

print("=== 全体性能サマリー ===")
if summary['overall_performance']:
    perf = summary['overall_performance']
    print(f"RMSE: {perf['rmse_mm']:.2f} mm")
    print(f"MAE: {perf['mae_mm']:.2f} mm")
    print(f"Median Error: {perf['median_error_mm']:.2f} mm")
    print(f"P95 Error: {perf['p95_error_mm']:.2f} mm")
    if perf.get('gain_percent'):
        print(f"Improvement Gain (vs Baseline-0): {perf['gain_percent']:.1f}%")

print("\n=== 主要な発見事項 ===")
for finding in summary['key_findings']:
    print(f"• {finding}")

In [ ]:
# 分割別性能比較
if summary['split_comparison']:
    print("\n=== 分割別性能比較 ===")
    
    # DataFrameで整理して表示
    comparison_data = []
    for split, metrics in summary['split_comparison'].items():
        comparison_data.append({
            'Split': split.capitalize(),
            'RMSE (mm)': f"{metrics['rmse']:.3f}",
            'MAE (mm)': f"{metrics['mae']:.3f}"
        })
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))

In [ ]:
# 詳細メトリクス表示（テストセット）
if 'test' in evaluation_results['detailed_results']:
    test_results = evaluation_results['detailed_results']['test']
    
    print("\n=== テストセット詳細メトリクス ===")
    test_metrics = test_results['metrics']
    
    print(f"RMSE: {test_metrics['rmse_mm']:.3f} mm")
    print(f"MAE: {test_metrics['mae_mm']:.3f} mm")
    print(f"Median Error: {test_metrics['median_error_mm']:.3f} mm")
    print(f"P90 Error: {test_metrics['p90_error_mm']:.3f} mm")
    print(f"P95 Error: {test_metrics['p95_error_mm']:.3f} mm")
    print(f"P99 Error: {test_metrics['p99_error_mm']:.3f} mm")
    print(f"Max Error: {test_metrics['max_error_mm']:.3f} mm")
    
    print("\n座標別統計:")
    print(f"X方向 MAE: {test_metrics['mae_x_mm']:.3f} mm")
    print(f"Y方向 MAE: {test_metrics['mae_y_mm']:.3f} mm")
    print(f"X方向 RMSE: {test_metrics['rmse_x_mm']:.3f} mm")
    print(f"Y方向 RMSE: {test_metrics['rmse_y_mm']:.3f} mm")

In [ ]:
# 統計的分析結果
if 'statistical_analysis' in test_results:
    stat_analysis = test_results['statistical_analysis']
    
    print("\n=== 統計的分析 ===")
    
    # 残差統計
    residual_stats = stat_analysis['residual_statistics']
    print("残差統計:")
    print(f"X方向: Mean={residual_stats['mean'][0]:.4f}, Std={residual_stats['std'][0]:.4f}")
    print(f"Y方向: Mean={residual_stats['mean'][1]:.4f}, Std={residual_stats['std'][1]:.4f}")
    
    # 正規性検定
    normality = stat_analysis['normality_tests']
    print("\n正規性検定 (Shapiro-Wilk):")
    for coord in ['x', 'y']:
        key = f'{coord}_normal'
        if key in normality:
            test = normality[key]
            status = "正規分布" if test['is_normal'] else "非正規分布"
            print(f"{coord.upper()}方向: p={test['p_value']:.6f} ({status})")
    
    # 相関分析
    correlations = stat_analysis['correlations']
    print("\n予測-実測相関:")
    for coord in ['x', 'y']:
        key = f'{coord}_correlation'
        if key in correlations:
            corr = correlations[key]
            print(f"{coord.upper()}方向: R={corr['r']:.4f}, R²={corr['r_squared']:.4f}")

In [ ]:
# ケース別分析結果
if 'case_analysis' in test_results and test_results['case_analysis']:
    case_analysis = test_results['case_analysis']
    
    print("\n=== ケース別性能分析 ===")
    
    # サマリー情報
    if '_summary' in case_analysis:
        case_summary = case_analysis['_summary']
        print(f"最良ケース: {case_summary['best_case']}")
        print(f"最悪ケース: {case_summary['worst_case']}")
        
        variability = case_summary['case_variability']
        print(f"ケース間誤差標準偏差: {variability['mean_error_std']:.3f} mm")
        print(f"ケース間誤差範囲: {variability['mean_error_range']:.3f} mm")
    
    # 個別ケース詳細（上位5ケースと下位5ケース）
    case_stats = {k: v for k, v in case_analysis.items() if k != '_summary'}
    
    if case_stats:
        # 平均誤差でソート
        sorted_cases = sorted(case_stats.items(), key=lambda x: x[1]['mean_error'])
        
        print("\n最良5ケース:")
        print("Case ID\t\tMean Error\tMax Error\tSamples")
        print("-" * 50)
        for case_id, stats in sorted_cases[:5]:
            print(f"{case_id[:12]:<12}\t{stats['mean_error']:.3f}\t\t{stats['max_error']:.3f}\t\t{stats['num_samples']}")
        
        print("\n最悪5ケース:")
        print("Case ID\t\tMean Error\tMax Error\tSamples")
        print("-" * 50)
        for case_id, stats in sorted_cases[-5:]:
            print(f"{case_id[:12]:<12}\t{stats['mean_error']:.3f}\t\t{stats['max_error']:.3f}\t\t{stats['num_samples']}")

## 複数モデル比較評価（オプション）

異なる実験設定で学習したモデルの比較を行います。

In [ ]:
# 比較対象モデルの設定（学習済みモデルがある場合のみ）
comparison_models = []

# ベースライン
baseline_config = get_baseline_config()
baseline_exp_manager = ExperimentManager(baseline_config)
baseline_model_path = baseline_exp_manager.exp_dir / "models" / "best_model.pth"
if baseline_model_path.exists():
    comparison_models.append({
        'name': 'Baseline MLP',
        'config': baseline_config,
        'model_path': str(baseline_model_path)
    })

# CVAE
cvae_config = get_cvae_config()
cvae_exp_manager = ExperimentManager(cvae_config)
cvae_model_path = cvae_exp_manager.exp_dir / "models" / "best_model.pth"
if cvae_model_path.exists():
    comparison_models.append({
        'name': 'CVAE',
        'config': cvae_config,
        'model_path': str(cvae_model_path)
    })

print(f"比較対象モデル数: {len(comparison_models)}")
for model in comparison_models:
    print(f"  - {model['name']}")

In [ ]:
# モデル比較実行（複数モデルがある場合）
if len(comparison_models) > 1:
    print("\n=== モデル比較評価開始 ===")
    
    comparator = ModelComparator(exp_manager)
    
    try:
        comparison_results = comparator.compare_models(
            comparison_models, datamodule
        )
        
        print("\n=== モデル比較完了 ===")
        
        # 比較結果表示
        individual_results = comparison_results['individual_results']
        comparison_analysis = comparison_results['comparison_analysis']
        
        # 性能ランキング表示
        print("\n=== 性能ランキング ===")
        for metric in ['rmse_mm', 'mae_mm']:
            if metric in comparison_analysis['performance_ranking']:
                ranking = comparison_analysis['performance_ranking'][metric]
                print(f"\n{metric.upper()} ランキング:")
                for i, entry in enumerate(ranking, 1):
                    if entry['value'] is not None:
                        print(f"  {i}. {entry['model']}: {entry['value']:.3f}")
        
        # 最良モデル表示
        print("\n=== 最良モデル ===")
        for metric, best in comparison_analysis['best_models'].items():
            print(f"{metric}: {best['model']} ({best['value']:.3f})")
            
    except Exception as e:
        print(f"\n❌ モデル比較エラー: {e}")
        comparison_results = None
else:
    print("\n複数の学習済みモデルが見つからないため、比較評価をスキップします。")
    comparison_results = None

## 評価完了

モデルの評価が完了しました。

### 保存された内容
- ✅ 詳細評価結果（detailed_results.json）
- ✅ 評価サマリー（evaluation_summary.json）
- ✅ 評価レポート（evaluation_report.md）
- ✅ 予測結果（predictions/）
- ✅ モデル比較結果（model_comparison/）

### 次のステップ
1. `04_analysis.ipynb` で結果の可視化・詳細分析
2. 性能改善のための設定調整
3. 追加実験の計画

### 評価結果の要約

In [ ]:
# 最終評価サマリー
print("=== 最終評価サマリー ===")
print(f"実験ID: {config.exp_id}")
print(f"モデルタイプ: {config.model.model_type}")

if 'test' in evaluation_results['detailed_results']:
    test_metrics = evaluation_results['detailed_results']['test']['metrics']
    print(f"\nテストセット性能:")
    print(f"  RMSE: {test_metrics['rmse_mm']:.2f} mm")
    print(f"  MAE: {test_metrics['mae_mm']:.2f} mm")
    print(f"  P95 Error: {test_metrics['p95_error_mm']:.2f} mm")
    
    if test_metrics.get('gain_percent'):
        print(f"  Improvement Gain (vs Baseline-0): {test_metrics['gain_percent']:.1f}%")

print(f"\n評価結果保存先: {exp_manager.exp_dir / 'evaluation'}")
print(f"予測結果保存先: {exp_manager.exp_dir / 'predictions'}")

if comparison_results:
    print(f"モデル比較結果: {exp_manager.exp_dir / 'model_comparison'}")

print("\n✅ 評価完了 - 04_analysis.ipynbで詳細分析を実行してください")